# Hypothesis Testing: SVM and Gradient Boost

## 1- Comparing results of SVM and Gradient Boost classifiers

In the previous phase, we did get results and cross-validation scores for the SVM model, but we did not get the chance to finish training the Gradient Boost model. So, now that Gradient Boost did finish training, we compare its cross-validation scores to those of SVM. We notice that the mean score of SVM is slightly higher than that of Gradient Boost, and now is the time to see whether this difference has any significance.

For this, we use a regular t-test, since training the data on completely different models can be considered to be independent, so we assume that the lists of CV scores obtained are independent. Our hypothesis $H_0$ is then that the difference in scores is not significant, and we set the significance level $\alpha$ to be 0.01.

Since a few folds failed to train for both models, their score in the lists is `nan` and needs to be removed before computing the t-test's p-value.

In [1]:
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
#from mlxtend.evaluate import paired_ttest_5x2cv
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
#comparing svm with gradient boost, w/o feature selection
# svm does look to have better results, the t-test will tell us whether this is actually significant

with open("scores_gb.pickle", "rb") as f1, open("clf.pickle", "rb") as f2:
    gb_scores = pickle.load(f1)
    svm_clf = pickle.load(f2)
    idx = svm_clf.best_index_
    svm_scores = [svm_clf.cv_results_[f"split{i}_test_score"][idx] for i in range(10)]

In [3]:
from scipy.stats import ttest_ind

#remove nans
gb_scores_ = [x for x in gb_scores if not np.isnan(x)]
svm_scores_ = [x for x in svm_scores if not np.isnan(x)]

print("Mean of Gradient Boost scores:\t", np.mean(gb_scores_))
print("Mean of SVM scores:\t\t", np.mean(svm_scores_))

t, p = ttest_ind(gb_scores_, svm_scores_)
print("p-value:\t\t\t", p)

Mean of Gradient Boost scores:	 0.3232376922446489
Mean of SVM scores:		 0.350712814335626
p-value:			 0.00015273342072732782


We get a p-value of 0.0001 which is less than $\alpha$ = 0.01, then we can reject $H_0$ with 99% confidence, and say that we are 99% confident that SVM's perceived improvements over GradientBoost are statistically significant.

The better model among these two is then the SVM model, for which we now conduct further t-tests.

## 2- Hypothesis testing for SVM: Impact of MLSMOTE, Grid Search, Feature Selection

Our objective here is to see whether MLSMOTE and Feature Selection provide statistically significant improvements to our SVM's performance. Recall that in the last project phase, we saw that scores showed a slight improvement with MLSMOTE, now is the time to determine whether this change in scores is statistically significant, or just an act of chance.

For this, we take the cross-validation scores of SVM both before and after applying MLSMOTE (scores available from Phase 3's work), and this time we use a Paired Student's t-test to test whether the means of the scores are statistically the same or not. We used this particular test because the two score sets are dependent this time (same model and almost same data).

For each score set, we needed to remove a few "nan" values, since some folds did not have samples from every single class, which causes SVM model fitting to fail. Most scores are not nan though, so we still have enough samples for the t-test to be relevant.

In this problem, the null hypothesis $H_0$ claims that no significant difference is seen between the scores before and after. We use $\alpha$=0.01, so that we can use the obtained p-value to make a claim at 99% confidence level.

We then repeat the same process to compare improvement before and after Grid Search, and before and after Feature Selection.

### a- With vs. Without MLSMOTE

In [5]:
with open("clf.pickle", "rb") as f1, open("clf_svm.pickle", "rb") as f2:
    svm = pickle.load(f1)
    svm_mlsmote = pickle.load(f2)

In [6]:
idx = svm.best_index_
idx_mlsmote = svm_mlsmote.best_index_
cv_scores_svm = [svm.cv_results_[f"split{i}_test_score"][idx] for i in range(10)]
cv_scores_svm_mlsmote = [svm_mlsmote.cv_results_[f"split{i}_test_score"][idx_mlsmote] for i in range(10)]

In [7]:
from scipy.stats import ttest_rel

#remove nans
scores_no_mlsmote = [x for x in cv_scores_svm if not np.isnan(x)]
scores_mlsmote = [x for x in cv_scores_svm_mlsmote if not np.isnan(x)]

t, p = ttest_rel(scores_no_mlsmote, scores_mlsmote)

In [8]:
p

0.0013491537095954922

For alpha = 0.01, we have p-value < $\alpha$, So, we can reject the null hypothesis $H_0$ with a confidence of 99%, and we can be 99% percent confident that applying MLSMOTE does improve performance, even if the performance numbers themselves are not great.

### b- Same t-test, but comparing with and without grid search

In [9]:
with open("svm_nogrid_scores.pickle", "rb") as f:
    cv_scores_svm_nogrid = pickle.load(f)
    scores_nogrid = [x for x in cv_scores_svm_nogrid if not np.isnan(x)][:-1]
    t, p = ttest_rel(scores_no_mlsmote, scores_nogrid)
    print("p-value:", p)

p-value: 0.005649038016994078


The obtained p-value is 0.005, less than $\alpha$ = 0.01. So, we can reject the null hypothesis $H_0$ with a confidence of 99%, and say that we are 99% confident that hyperparameter tuning using grid search does improve our SVM's performance.

### c- Same t-test, for feature selection this time

In [10]:
with open("clf_fs.pickle", "rb") as f:
    svm_fs = pickle.load(f)
    idx_fs = svm_fs.best_index_
    cv_scores_svm_fs = [svm_fs.cv_results_[f"split{i}_test_score"][idx_fs] for i in range(10)][:-1]
    
scores_fs = [x for x in cv_scores_svm_fs if not np.isnan(x)]
t, p = ttest_rel(scores_no_mlsmote, scores_fs)
print("p-value:", p)

p-value: 0.46890355576400666


This time, we obtain a p-value of 0.468 which is much greater than $\alpha$=0.01, thus we fail to reject the null hypothesis $H_0$ at a 99% confidence level.